In [ ]:
from dldd import ClassificationModel
from pytorch_lightning import Trainer
from torch_geometric.data import DataLoader
from dldd.utils import TwoGraphData
import torch
import pickle

In [ ]:
with open('data/train.pkl', 'rb') as file:
    train = pickle.load(file)
with open('data/val.pkl', 'rb') as file:
    val = pickle.load(file)
with open('data/test.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
# create DataLoader
train_dl = DataLoader(train,
                batch_size=256,
                num_workers=4,
                follow_batch=['protein_x', 'drug_x'],
                shuffle=True)
val_dl = DataLoader(val,
                batch_size=256,
                num_workers=4,
                follow_batch=['protein_x', 'drug_x'],
                shuffle=True)
test_dl = DataLoader(test,
                batch_size=256,
                num_workers=4,
                follow_batch=['protein_x', 'drug_x'],
                    shuffle=True)

In [ ]:
# create the model
model = ClassificationModel()

# train the model
trainer = Trainer(gpus=1, max_epochs=100, log_every_n_steps=10)
trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)
trainer.test(model, test_dataloaders=test_dl)